In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input, Model
import numpy as np

# verify if GPU is available
print(tf.test.is_gpu_available())

# set memory growth to true
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Setting memory growth to True for GPU: ", physical_devices[0])
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# dont display much info of tensorflow
tf.get_logger().setLevel('ERROR')


2023-12-29 12:20:02.248725: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-29 12:20:02.333601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-29 12:20:02.333648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-29 12:20:02.336104: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-29 12:20:02.351989: I tensorflow/core/platform/cpu_feature_guar

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
Setting memory growth to True for GPU:  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


2023-12-29 12:20:04.599947: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-29 12:20:04.669536: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-29 12:20:04.669716: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:

# Load data
manta_path = "/home/vm/SSL_Project_1/data/processed/bag_2023-07-04_15-23-48/_manta.npy"
xiris_path = "/home/vm/SSL_Project_1/data/processed/bag_2023-07-04_15-23-48/_xiris.npy"
y_path = "/home/vm/SSL_Project_1/data/processed/bag_2023-07-04_15-23-48/_y.npy"
feats_path = "/home/vm/SSL_Project_1/data/processed/bag_2023-07-04_15-23-48/_feats.npy"

# load numpy arrays and display shapes
manta = np.load(manta_path)
xiris = np.load(xiris_path)
y = np.load(y_path)
print("manta shape: ", manta.shape)
print("xiris shape: ", xiris.shape)
print("y shape: ", y.shape) # laser power and velocity

#feats = np.load(feats_path)
#print("feats shape: ", feats.shape)
y = y[:, 0] # only use laser power
print("y shape: ", y.shape)

# normalize y
y = y / np.max(y)

manta shape:  (9587, 320, 320)
xiris shape:  (9587, 320, 320)
y shape:  (9587, 2)
y shape:  (9587,)


In [3]:
# unique values in y
y_unique = np.unique(y)
print("unique values in y: ", y_unique)

# encode y as integers based on unique values
y_encoded = np.zeros(y.shape)
for i in range(len(y_unique)):
    y_encoded[y == y_unique[i]] = i
print("y encoded: ", y_encoded)

unique values in y:  [0.18181818 0.45454545 0.72727273 1.        ]
y encoded:  [0. 0. 0. ... 3. 3. 3.]


In [4]:
# split data into train and test (manta as input and y as output) with shuffle as true
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(manta, y_encoded, test_size=0.2, shuffle=True)
print("x_train shape: ", x_train.shape)
print("y_train shape: ", y_train.shape)
print("x_test shape: ", x_test.shape)
print("y_test shape: ", y_test.shape)


x_train shape:  (7669, 320, 320)
y_train shape:  (7669,)
x_test shape:  (1918, 320, 320)
y_test shape:  (1918,)


In [5]:
def create_encoder():
    inputs = Input(shape=(320, 320, 1))
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.Flatten()(x)
    model = Model(inputs=inputs, outputs=x)
    return model

# add projection head
def create_projection_head():
    model = models.Sequential()
    model.add(layers.Dense(128, activation='relu'))
    return model

input_shape = (320, 320, 1)

def add_projection_head(encoder):
    inputs = Input(shape=input_shape)
    features = encoder(inputs)
    outputs = layers.Dense(128, activation='relu')(features)
    model = Model(inputs=inputs, outputs=outputs)
    return model


encoder = create_encoder()
encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 320, 320, 1)]     0         
                                                                 
 model (Functional)          (None, 369664)            55744     
                                                                 
 dense (Dense)               (None, 128)               47317120  
                                                                 
Total params: 47372864 (180.71 MB)
Trainable params: 47372864 (180.71 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2023-12-29 12:20:14.098408: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-29 12:20:14.098658: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-29 12:20:14.098748: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
import tensorflow_addons as tfa
from tensorflow import keras

class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super().__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Normalize feature vectors
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normalized, tf.transpose(feature_vectors_normalized)
            ),
            self.temperature,
        )
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)

/home/vm/laser/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [7]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super().__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Normalize feature vectors
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normalized, tf.transpose(feature_vectors_normalized)
            ),
            self.temperature,
        )

        # Assuming labels is already one-dimensional
        labels = tf.cast(labels, dtype=tf.int32)
        labels = tf.reshape(labels, shape=(-1,))

        # Use native TensorFlow implementation of npairs_loss
        return tf.nn.sparse_softmax_cross_entropy_with_logits(labels, logits)


In [8]:
# complile model with SupervisedContrastiveLoss
encoder_with_projection_head.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=SupervisedContrastiveLoss(temperature=0.1),
)

In [9]:
# print manta and y_encoded shapes
print("manta shape: ", manta.shape)
print("y_encoded shape: ", y_encoded.shape)

manta shape:  (9587, 320, 320)
y_encoded shape:  (9587,)


In [10]:
# train model with manta an y_encoded
encoder_with_projection_head.fit(
    x=x_train,
    y=y_train,
    batch_size=32,
    epochs=10,
)

Epoch 1/10


2023-12-29 12:20:23.040848: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-29 12:20:23.143806: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-29 12:20:23.557712: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-29 12:20:25.264734: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f72debcc7e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-29 12:20:25.264763: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2023-12-29 12:20:25.269504: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1703852425.428414    6938 device_compiler.h:186] 

240/240 [==============================] - 14s 37ms/step - loss: 3.4707
Epoch 2/10
240/240 [==============================] - 8s 33ms/step - loss: 3.4657
Epoch 3/10
240/240 [==============================] - 8s 33ms/step - loss: 3.4656
Epoch 4/10
240/240 [==============================] - 8s 33ms/step - loss: 3.4654
Epoch 5/10
240/240 [==============================] - 8s 33ms/step - loss: 3.4650
Epoch 6/10
240/240 [==============================] - 8s 33ms/step - loss: 3.4650
Epoch 7/10
240/240 [==============================] - 8s 33ms/step - loss: 3.4650
Epoch 8/10
240/240 [==============================] - 8s 33ms/step - loss: 3.4650
Epoch 9/10
240/240 [==============================] - 8s 33ms/step - loss: 3.4650
Epoch 10/10
240/240 [==============================] - 8s 33ms/step - loss: 3.4650


In [11]:
a

NameError: name 'a' is not defined

In [12]:
a = tf.constant([[1, 2], [3, 4],[5, 6]], dtype=tf.float16)
b = tf.constant([[5, 9], [3, 6],[1, 8]], dtype=tf.float16)
y_pred = tf.linalg.norm(a - b, axis=1)
y_pred

<tf.Tensor: shape=(3,), dtype=float16, numpy=array([8.06 , 2.   , 4.473], dtype=float16)>

In [ ]:


# Create two instances of the base network
base_network = create_base_network()
image1_network = models.Model(inputs=base_network.input, outputs=base_network.output)
image2_network = models.Model(inputs=base_network.input, outputs=base_network.output)

# Define the contrastive loss function
class ContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, laser_power1, velocity1, laser_power2, velocity2):
        # Calculate the margins as functions of laser power and velocity
        margin_laser_power = tf.abs(laser_power1 - laser_power2)
        margin_velocity = tf.abs(velocity1 - velocity2)

        square_pred = tf.square(y_pred)
        margin_square_laser_power = tf.square(tf.maximum(margin_laser_power - y_pred, 0))
        margin_square_velocity = tf.square(tf.maximum(margin_velocity - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * (margin_square_laser_power + margin_square_velocity))
    
# Create input layers for laser power and velocity
laser_power_input1 = layers.Input(shape=(1,))
velocity_input1 = layers.Input(shape=(1,))
laser_power_input2 = layers.Input(shape=(1,))
velocity_input2 = layers.Input(shape=(1,))

# Connect input layers to base networks
image1_output = image1_network(base_network.output)
image2_output = image2_network(base_network.output)

# Concatenate image and feature outputs
merged_output = layers.concatenate([image1_output, image2_output, laser_power_input1, velocity_input1, laser_power_input2, velocity_input2])


# Compile the model with the contrastive loss function
model = models.Model(inputs=[image1_network.input, image2_network.input, laser_power1, velocity1, laser_power2, velocity2], outputs=distance)
model.compile(optimizer='adam', loss=ContrastiveLoss())


def calculate_similarity_indicator(laser_power1, velocity1, laser_power2, velocity2):
    # Calculate the difference in laser power and velocity
    difference_laser_power = abs(laser_power1 - laser_power2)
    difference_velocity = abs(velocity1 - velocity2)

    # Define a threshold for considering laser power and velocity as 'the same'
    threshold = 0.01

    # If the differences are below the threshold, return 1, else return 0
    if difference_laser_power < threshold and difference_velocity < threshold:
        return 1
    else:
        return 0

# Assuming you have a binary indicator representing whether the laser power and velocity are the same for each pair of images
similarity_indicator = calculate_similarity_indicator(laser_power1, velocity1, laser_power2, velocity2)

# Train the model
history = model.fit([image1, image2, laser_power1, velocity1, laser_power2, velocity2], similarity_indicator, epochs=10)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

2023-12-28 18:02:17.636605: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-28 18:02:17.720150: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-28 18:02:17.720203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-28 18:02:17.723229: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-28 18:02:17.738875: I tensorflow/core/platform/cpu_feature_guar

In [ ]:


# Assuming you have your images and features preprocessed and stored in numpy arrays
# image1, image2, laser_power, velocity

# Define the base network architecture
def create_base_network():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32))
    return model

# Create two instances of the base network
base_network = create_base_network()
image1_network = models.Model(inputs=base_network.input, outputs=base_network.output)
image2_network = models.Model(inputs=base_network.input, outputs=base_network.output)

# Define the distance layer
class DistanceLayer(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, image1, image2, laser_power1, velocity1, laser_power2, velocity2):
        image_distance = tf.reduce_sum(tf.square(image1 - image2), axis=-1)
        laser_power_distance = tf.square(laser_power1 - laser_power2)
        velocity_distance = tf.square(velocity1 - velocity2)
        return image_distance + laser_power_distance + velocity_distance
    

class ContrastiveLossLaserPower(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, laser_power1, laser_power2):
        # Calculate the margin as a function of laser power
        margin_laser_power = tf.abs(laser_power1 - laser_power2)

        square_pred = tf.square(y_pred)
        margin_square_laser_power = tf.square(tf.maximum(margin_laser_power - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square_laser_power)

class ContrastiveLossVelocity(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, velocity1, velocity2):
        # Calculate the margin as a function of velocity
        margin_velocity = tf.abs(velocity1 - velocity2)

        square_pred = tf.square(y_pred)
        margin_square_velocity = tf.square(tf.maximum(margin_velocity - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square_velocity)
    
class ContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, laser_power1, velocity1, laser_power2, velocity2):
        # Calculate the margins as functions of laser power and velocity
        margin_laser_power = tf.abs(laser_power1 - laser_power2)
        margin_velocity = tf.abs(velocity1 - velocity2)

        square_pred = tf.square(y_pred)
        margin_square_laser_power = tf.square(tf.maximum(margin_laser_power - y_pred, 0))
        margin_square_velocity = tf.square(tf.maximum(margin_velocity - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * (margin_square_laser_power + margin_square_velocity))




# Create the distance layer
distance = distance_layer(image1_network.output, image2_network.output, laser_power1, velocity1, laser_power2, velocity2)
model = models.Model(inputs=[image1_network.input, image2_network.input, laser_power1, velocity1, laser_power2, velocity2], outputs=distance)

# Compile the model
model.compile(optimizer='adam', loss='contrastive_loss')  # You need to define 'contrastive_loss'

# Train the model
# model.fit([image1, image2, laser_power, velocity], labels, epochs=10)


2023-12-28 18:02:39.564482: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-28 18:02:39.667648: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-28 18:02:39.667897: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

NameError: name 'distance_layer' is not defined

In [ ]:
class CombinedLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=0.5):
        super().__init__()
        self.alpha = alpha
        self.contrastive_loss_laser_power = ContrastiveLossLaserPower()
        self.contrastive_loss_velocity = ContrastiveLossVelocity()

    def call(self, y_true, y_pred, laser_power1, laser_power2, velocity1, velocity2):
        loss_laser_power = self.contrastive_loss_laser_power(y_true, y_pred, laser_power1, laser_power2)
        loss_velocity = self.contrastive_loss_velocity(y_true, y_pred, velocity1, velocity2)
        return self.alpha * loss_laser_power + (1 - self.alpha) * loss_velocity


In [ ]:
class ContrastiveLossLaserPower(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, laser_power1, laser_power2):
        # Calculate the margin as a function of laser power
        margin_laser_power = tf.abs(laser_power1 - laser_power2)

        square_pred = tf.square(y_pred)
        margin_square_laser_power = tf.square(tf.maximum(margin_laser_power - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square_laser_power)

class ContrastiveLossVelocity(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred, velocity1, velocity2):
        # Calculate the margin as a function of velocity
        margin_velocity = tf.abs(velocity1 - velocity2)

        square_pred = tf.square(y_pred)
        margin_square_velocity = tf.square(tf.maximum(margin_velocity - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square_velocity)
    
model.compile(optimizer='adam', loss=[ContrastiveLossLaserPower(), ContrastiveLossVelocity()])

NameError: name 'model' is not defined